# Opulent Voice Numerology

For now, the opv-cxx-demod package supports a single fixed set of parameter choices (one "numerology"). This notebook computes them and documents the process of choosing them.

## Nomenclature
`Opus` is the chosen vocoder; `OPV` is short for ***Opulent Voice*** (see what we did there?) and refers to the overall framing, coding, and modulation scheme.

Following M17, we define the following types of bits:

| Type  | Description |
|-------|-----------------------------------------------|
| type1 | initial data bits |
| type2 | bits after encoding |
| type3 | encoded bits after puncturing (if applicable) |
| type4 | decorrelated and interleaved type3 bits |


In [ ]:
import math
import sympy

### Vocoder Bit Rate

`opus_bitrate` is the fixed bit rate the vocoder is configured to use. For now, we are operating Opus in CBR (constant bit rate) mode, so it always uses exactly this bit rate. It would also be possible to operate Opus in VBR (variable bit rate) mode, in which case this becomes the maximum allowed bit rate (and any unused bits can be made available for other purposes).
The bit rate is chosen to achieve the desired voice quality. Opus recommends a bit rate of 16k to 20k for "Wideband" audio (8 kHz bandwidth).

In [ ]:
opus_bitrate = 16000    # bits/second

### Vocoder Frame Duration

`opus_frame_duration` is the length of the vocoder frame in seconds. This must be chosen from Opus's list of supported frame sizes: 2.5, 5, 10, 20, 40, or 60 ms.

In [ ]:
opus_frame_duration = 20 * 0.001
opus_frame_type1_bytes = int(opus_frame_duration * opus_bitrate) / 8
assert opus_frame_type1_bytes == int(opus_frame_type1_bytes)
opus_frame_type1_bytes = int(opus_frame_type1_bytes)
opus_frame_duration, opus_frame_type1_bytes

### Vocoder Frames per Channel Frame

`opus_frames_per_opv_frame` is the number of vocoder frames packed into each stream-mode frame defined for Opulent Voice.

In [ ]:
opus_frames_per_opv_frame = 2
opv_frame_duration = opus_frames_per_opv_frame * opus_frame_duration
opv_frame_duration

### PCM Sample Rate
The input to the voice encoder and the output from the voice decoder are raw streams of 16-bit unsigned integer samples, single channel, at a predetermined sample rate. This rate might vary from implementation to implementation, but 8000 samples/second is a typical value for speech. The rest of the numerology does not depend on the PCM sample rate.

In [ ]:
audio_sample_rate = 8000    # 16-bit samples/second
audio_samples_per_frame = int(audio_sample_rate * opus_frame_duration)
audio_samples_per_frame

### Physical Layer Header
For now, the physical header is a black box of a certain size.

In [ ]:
plheader_type1_bytes = 15   # multiples of 3 fill the Golay code
plheader_type1_bits = plheader_type1_bytes * 8
plheader_type2_bits = plheader_type1_bits * 2   # 12,24 Golay codes, rate 1/2
plheader_type3_bits = plheader_type2_bits   # no puncturing for plheader
plheader_type3_bits

### Payload (Vocoder Data in Stream Mode)

In [ ]:
raw_payload_bits = opus_bitrate * opv_frame_duration
payload_type1_bits = raw_payload_bits + 4   # convolutional encoder tail
payload_type2_bits = payload_type1_bits * 2 # Rate 1/2 convolutional code

# Puncturing is 11/12, in this pattern: 111111111110, so we remove one bit
# for every whole multiple of 12 type2 bits.
num_punctured_bits = math.floor(payload_type2_bits/12)

# The result may be an inconvenient number. We'd like it to be a multiple of 8.
# So, after puncturing, we may add a few zero bits of padding.
payload_type3_bits = math.ceil((payload_type2_bits - num_punctured_bits)/8)*8
payload_type3_padding = int(payload_type3_bits - (payload_type2_bits - num_punctured_bits))
payload_type3_bits, payload_type3_padding

### Combined Type 3 Frame

In [ ]:
opv_frame_type3_bits = plheader_type3_bits + payload_type3_bits
opv_frame_type3_bytes = opv_frame_type3_bits / 8
assert opv_frame_type3_bits == opv_frame_type3_bytes * 8
opv_frame_type3_bytes

### Transmitted Symbol Rate for Voice Stream Mode
The combined type3 frame is prefixed with a 16-bit sync word. The transmitted symbol rate is chosen so that the prefixed frames come out at the correct frame rate.

In [ ]:
opv_bit_rate = (16 + opv_frame_type3_bits) / opv_frame_duration
opv_symbol_rate = opv_bit_rate / 2  # 4FSK, so 2 bits per symbol
opv_symbol_rate

## BERT Mode
To facilitate testing, we have a Bit Error Rate Testing (BERT) mode. A fixed pseudorandom sequence is broken up into frames, with no header added. Thus, the BERT data takes up the space that would be allocated to the plheader as well as the payload in a stream-mode voice frame. A prime number is chosen as the number of bits in each frame, so that each frame is unique over a relatively long period of time.

The BERT data is convolutionally encoded and punctured in the same way as payload data in stream-mode voice. In order to make this convenient, up to three additional encoder tail bits may be added to bring the encoder input to an integer number of bytes. Then, after puncturing, further padding bits may be needed to bring the BERT-mode frame size to exactly match the stream-mode voice frame size.

So, the prime number chosen is is the largest prime that will result in a type3 frame size less than or equal to that of a stream-mode voice frame.

In [ ]:
desired_type3_size = opv_frame_type3_bits

# approximate the type2 size by undoing the puncture ratio
est_type2_size = math.floor((12/11) * desired_type3_size)

# the actual type2 size must be a multiple of 16, since we want the
# type1 size (including the encoder tail) to be an integer number
# of bytes and then the encoder doubles the length
bert_type2_bits = est_type2_size - (est_type2_size % 16)

# Puncturing is 11/12, in this pattern: 111111111110, so we remove one bit
# for every whole multiple of 12 type2 bits.
num_punctured_bits = math.floor(bert_type2_bits/12)
bert_punctured_size = bert_type2_bits - num_punctured_bits
bert_postpuncture_padding = desired_type3_size - bert_punctured_size

type1_size = int(est_type2_size/2) - 4   # un-encode and remove tail bits
bert_preencode_padding = 0
while not sympy.isprime(type1_size):
    type1_size -= 1
    bert_preencode_padding += 1
bert_bits_per_frame = type1_size

(bert_bits_per_frame, bert_preencode_padding, bert_postpuncture_padding, bert_type2_bits)

## Creating Numerology.h
The following can be copied and pasted into source file `Numerology.h` to define everything we've calculated. Names have been kept from the current draft source code, though they are not wholely consistent.

In [ ]:
print(f'    const int opus_bitrate = {opus_bitrate};')
print(f'    const int encoded_plheader_size = {plheader_type3_bits};')
print(f'    const int punctured_payload_size = {payload_type3_bits};')
print(f'    const int frame_size_bits = {opv_frame_type3_bits};')
print(f'    const int frame_size_bytes = {opv_frame_type3_bytes};')
print(f'    const int audio_sample_rate = {audio_sample_rate};')
print(f'    const int audio_frame_size = {audio_samples_per_frame};')
print(f'    const int opus_frame_size_bytes = {opus_frame_type1_bytes};')
# wrong in C++ code! print(f'    const int audio_payload_bits = {};')
print(f'    const int symbol_rate = {opv_symbol_rate};')
print(f'    const int bert_bits_per_frame = {bert_bits_per_frame};')
print(f'    const int bert_extra_bits = {bert_preencode_padding};')
print(f'    const int bert_encoded_size = {bert_type2_bits};')
print(f'    const int bert_punctured_size = {bert_punctured_size};')
print(f'    const int bert_postpuncture_padding = {bert_postpuncture_padding};')